In [1]:
# USER INPUT
city_name = 'santa_clara'
url = 'https://santaclara.legistar.com/Calendar.aspx'

In [2]:
# imports
import pandas as pd
import numpy as np
from urllib import parse
from urllib import request
import bs4 as bs

import selenium as se
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from tqdm import tqdm

# helper functions
def get_page_links(driver):
    pagelinks_xpath = "//td[@class='rgPagerCell NumericPages']/div[1]/a"
    pagelinks = driver.find_elements_by_xpath(pagelinks_xpath)
    pagelinks = pagelinks[:int(len(pagelinks)/2)]
    return [l.text for l in pagelinks], pagelinks

def extract_table(page_source, table_id):
    # find table in page
    soup = bs.BeautifulSoup(page_source)
    table = soup.select(table_id)[0]
    num_cols = int(table.td.get('colspan'))

    # extract column headers
    header_data = [''.join(cell.stripped_strings) for cell in table.find_all('th')]
    header_data = [h for h in header_data if h!='Data pager']
    assert(len(header_data)==num_cols)

    # extract text and URL data from table
    text_data, url_data = [], []
    for row in table.find_all('tr'):
        row_text, row_url = [], []
        for td in row.find_all('td'):
            row_text.append(''.join(td.stripped_strings))
            if td.find('a'):
                row_url.append(td.a.get('href'))
            else:
                row_url.append(np.nan)
            if len(row_text)==num_cols and len(row_url)==num_cols:
                text_data.append(row_text)
                url_data.append(row_url)
                
    # turn into dataframe
    num_cols = table.td.get('colspan')
    text_df = pd.DataFrame(text_data, columns=header_data)
    url_df = pd.DataFrame(url_data, columns=header_data)
    df = pd.merge(text_df, url_df, left_index=True, right_index=True, suffixes=(' Text', ' URL'))
    
    return df

In [3]:
# launch driver
driver = Firefox()
driver.get(url)

In [5]:
# MANUAL: select date range dropdown  to 'All Years'

In [6]:
# click through pages and save html
c = 1
page_data = []
while True:
    pages, pagelinks = get_page_links(driver)
    try:
        # click on the integer we want
        i = pages.index(str(c))
        link = pagelinks[i]
    except:
        # if it's not there and the list ends with '...', click on '...'
        if pages[-1]=='...':
            link = pagelinks[-1]
        # if it's not there and the list starts with '...', we are done.
        else:
            break
    link.click()
    input('Scrape page {}?'.format(c))
    page_data.append(driver.page_source)
    c += 1

Scrape page 1?
Scrape page 2?
Scrape page 3?
Scrape page 4?
Scrape page 5?


In [7]:
# extract table data
table_id = '#ctl00_ContentPlaceHolder1_gridCalendar_ctl00'
page_dfs = [extract_table(page, table_id) for page in tqdm(page_data)]

100%|██████████| 5/5 [00:01<00:00,  3.82it/s]


In [9]:
data = pd.concat(page_dfs)
len(data)

413

In [11]:
data.columns

Index(['Name Text', 'Meeting Date Text', ' Text', 'Meeting Time Text',
       'Meeting Location Text', 'Meeting Details Text', 'Agenda Text',
       'Minutes Text', 'Agenda Packet Text', 'Video or Audio Text',
       'Caption Notes Text', 'eComment Text', 'Name URL', 'Meeting Date URL',
       ' URL', 'Meeting Time URL', 'Meeting Location URL',
       'Meeting Details URL', 'Agenda URL', 'Minutes URL', 'Agenda Packet URL',
       'Video or Audio URL', 'Caption Notes URL', 'eComment URL'],
      dtype='object')

In [12]:
save_path = '../data/scraping/scraped_tables/{}.csv'.format(city_name)
data.to_csv(save_path)

In [12]:
# # generate results
# submit_button_id = 'ctl00_ContentPlaceHolder1_btnSearch'
# button = driver.find_element_by_id(submit_button_id)
# button.submit()